In [1]:
import subprocess
import os
import time
import threading
import IPython.display as display
import ipywidgets as widgets

import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt


In [2]:
# -----------------------
# SETTINGS
# -----------------------
script = "parallel_string_sizing.py"
log_file = "log.txt"

# -----------------------
# LAUNCH SCRIPT
# -----------------------
with open(log_file, "w") as logfile:
    process = subprocess.Popen(
        ["python", script],
        stdout=logfile,
        stderr=subprocess.STDOUT
    )

pid = process.pid
print(f"🚀 Script started with PID: {pid}. Output is logging to: {log_file}")

# -----------------------
# STOP BUTTON
# -----------------------
stop_btn = widgets.Button(description="🛑 Stop Script", button_style='danger')

def stop_script(b):
    process.terminate()
    print(f"🔴 Terminated script with PID {pid}")

stop_btn.on_click(stop_script)
display.display(stop_btn)

# -----------------------
# LIVE LOG DISPLAY (last 10 lines)
# -----------------------
log_output = widgets.Output()
display.display(log_output)

def update_log_display():
    while process.poll() is None:
        with log_output:
            log_output.clear_output(wait=True)
            try:
                with open(log_file, "r") as f:
                    lines = f.readlines()[-10:]
                    print("".join(lines))
            except:
                print("Waiting for log...")
        time.sleep(2)

# Start background thread to monitor log
threading.Thread(target=update_log_display, daemon=True).start()


🚀 Script started with PID: 44220. Output is logging to: log.txt


Button(button_style='danger', description='🛑 Stop Script', style=ButtonStyle())

Output()

In [3]:
# Load your simulation results
df = pd.read_csv("string_sizing_results.csv")

# Preview
df.sample(10)


,Module,Year,ModulesPerString,VocMax
472,LR5-72HBD-560M,2012,25,1297.35
398,LR5-72HBD-555M,2019,26,1375.90
101,LR5-72HBD-540M,2023,25,1269.59
330,LR5-72HBD-555M,2002,27,1405.42
210,LR5-72HBD-550M,1998,27,1388.15
306,LR5-72HBD-550M,2022,27,1432.16
564,LR5-72HBD-565M,2009,25,1316.56
469,LR5-72HBD-560M,2011,26,1356.97
616,LR5-72HBD-565M,2022,25,1338.01
251,LR5-72HBD-550M,2008,28,1453.45


In [4]:
df["ExceedsLimit"] = df["VocMax"] > 1500


In [5]:
# Create a pivot table of max Voc across all years
pivot = df.pivot_table(index='Module',
                       columns='ModulesPerString',
                       values='VocMax',
                       aggfunc='max')

# Optional: sort modules alphabetically
pivot = pivot.sort_index()


In [6]:
# Add a helper column to highlight rows with any Voc > 1500V
pivot["Any_Over_1500V"] = (pivot > 1500).any(axis=1)


In [7]:
pivot.to_csv("pivot_summary_by_module.csv") 
print("✅ Saved: pivot_summary_by_module.csv")
pivot.head(10)


✅ Saved: pivot_summary_by_module.csv


ModulesPerString,25,26,27,28,Any_Over_1500V
Module,,,,,
LR5-72HBD-540M,1324.90,1377.89,1430.89,1483.89,False
LR5-72HBD-545M,1328.99,1382.15,1435.31,1488.47,False
LR5-72HBD-550M,1333.06,1386.38,1439.71,1493.03,False
LR5-72HBD-555M,1337.32,1390.82,1444.31,1497.80,False
LR5-72HBD-560M,1341.26,1394.91,1448.57,1502.22,True
LR5-72HBD-565M,1345.28,1399.09,1452.90,1506.71,True


In [ ]:
# Load into DataFrame
import seaborn as sns
import matplotlib.pyplot as plt


# Set seaborn style
sns.set(style="darkgrid")

# FacetGrid with lineplot
g = sns.FacetGrid(df, col="Module", col_wrap=1, height=5, aspect=1.5)
g.map_dataframe(sns.lineplot, x="Year", y="VocMax", hue="ModulesPerString", palette="tab10", marker="o")

# Add legend and labels
g.add_legend(title="ModulesPerString")
g.set_titles(row_template="{row_name}")
g.set_axis_labels("Year", "VocMax")
plt.tight_layout()
plt.show()


In [ ]:
f, ax = plt.subplots(figsize=(7, 5))
sns.despine(f)

sns.histplot(
    df,
    x="VocMax", hue="ModulesPerString",
    multiple="stack",
    palette="light:m_r",
    edgecolor=".3",
    linewidth=.5,
    log_scale=True,
)
ax.xaxis.set_major_formatter(mpl.ticker.ScalarFormatter())
# ax.set_xticks([500, 1000, 2000, 5000, 10000])